In [8]:
# Jupyter의 경우 python 과 다르게 __fine__ 출력을 하면 다음과 같이 정의되지 않았다고 오류가 난다.

import os

print(__file__)

NameError: name '__file__' is not defined

In [10]:
# 따라서 os.path.abspath('')  절대경로를 이용하여 이미지 경로를 대입한다.

import os 
os.path.abspath('')

'c:\\Users\\admin\\Desktop\\Stereo'

In [11]:
import os
path_img = 'cat.jpg'
path_img = os.path.join(os.path.abspath(''),
                            path_img)
print(path_img)

c:\Users\admin\Desktop\Stereo\cat.jpg


In [12]:
# Introduction:
import os

import numpy as np
from PIL import Image

In [13]:
#!user/bin/env python
# -*- coding:utf-8 -*-

# --------------- 매개변수 부분 --------------- #

# Original Image Path：
path_img = 'cat.jpg'

# Projection Image Path :
path_proj = 'tac.jpg'

# Projection Image Output Size (Unit: Pixel)
w_proj = 400
h_proj = 300

# Offset(Unit : Percentage)
# 비고 : 출력된 투영 이미지의 중심 위치를 조정
offset_hor = 0  # Horizontal offset (positive to right）
offset_ver = 0.4  # Vertical offset (positive downward)

# Scaling multiple
scale = 1.5

# Angle of Rotation of Coordinate axis (좌표축의 회전 각도):
# Note : Rotation is to obtain different spherical projection conditions (reason/chestnut head)
# 회전은 서로 다른 구면 투영 상황을 얻기 위한 것이다. (밤머리)
alpha = 0 * np.pi / 180  # Angle of Rotation aroun the x-axis
beta = -5 * np.pi / 180  # Rotation angle around y-axis (around 150° to get chestnut head)
gamma = 0 * np.pi / 180  # angle of rotation about z axis

# --------------- 실현 --------------- #

In [14]:
def get_point_on_sphere(point: np.ndarray, r: float) -> np.ndarray:
    """ Calculate the coordinates of the intersection point P between a point Q on the z=0 plane
    and a projection point D on the sphere

    Args:
        point (np.ndarray): point Q coordinate
        r (float): sphere radius

    Returns:
        np.ndarray: Coordinates of the intersection point P on the sphere
    """
    [x, y, z] = point
    k = 2 * r**2 / (x**2 + y**2 + r**2)
    # Derivation and simplification of the obtained coefficients (see README.md for the derivation process)
    return np.array([k * x, k * y, (k - 1) * r], dtype=np.float32)


def axis_rotate(point: np.ndarray, rot_mat: np.ndarray) -> np.ndarray:
    """Calculate the change of point P coordinate after rotation of coordinate system

    Args:
        point (np.ndarray): point P coordinate
        rot_mat (np.ndarray): Rotation matrix (see README.md for derivation)

    Returns:
        np.ndarray: transformed point P coordinate
    """
    return np.dot(rot_mat, point)


def get_pix_on_img(point: np.ndarray, r: float, h_img: int,
                   w_img: int) -> tuple:
    """The inverse process of spherical projection, calculating the coordinates of a point P on the sphere on the original image.

    Args:
        point (np.ndarray): point P coordinate
        r (float): sphere radius
        h_img (int): original image height
        w_img (int): original image width

    Returns:
        tuple: Pixel coordinates corresponding to the original image
    """
    [x, y, z] = point
    if z > r:
        z = r
    row = np.arccos(z / r) / np.pi
    col = np.arctan2(y, x) / 2 / np.pi + 0.5  # Adding 0.5 is moving the center of the image to the plane y=0.
    # The coordinate range is restored to the original image size:
    row = round(row * h_img) % h_img
    col = round(col * w_img) % w_img
    return (row, col)


def projection(pix_proj: tuple, r: float, h_img: int, w_img: int, h_proj: int,
               w_proj: int) -> tuple:
    """spherical projection

    Args:
        pix_proj (tuple): pixel coordinates on projected images
        r (float): sphere radius
        h_img (int): original image height
        w_img (int): original image width
        h_proj (int): projected image height
        w_proj (int): projection width

    Returns:
        tuple: Pixel coordinates corresponding to the original image
    """
    # The pixel point coordinate on the projection image is converted to 3D coordinate:
    (row, col) = pix_proj
    x = row + (offset_ver - 0.5) * h_proj
    y = col + (offset_hor - 0.5) * w_proj
    z = 0
    Q = np.array([x, y, z], dtype=np.float32)
    P = get_point_on_sphere(Q, r)
    P = axis_rotate(P, rot_mat)
    return get_pix_on_img(P, r, h_img, w_img)


In [15]:
if __name__ == '__main__':
    

    path_img = os.path.join(os.path.abspath(''),
                            path_img)
    path_proj = os.path.join(os.path.abspath(''),
                             path_proj)

    arr_img = np.array(Image.open(path_img))
    arr_proj = np.zeros((h_proj, w_proj, 3), dtype=np.uint8)

    h_img = arr_img.shape[0]
    w_img = arr_img.shape[1]

    r = min(h_proj, w_proj) / 10 * scale  # The radius of the sphere affects how much content is presented on the projected image.

    # It's always automatically formatted to look so ugly.
    rot_mat = np.array([[
        np.cos(gamma) * np.cos(beta),
        np.cos(gamma) * np.sin(beta) * np.sin(alpha) -
        np.sin(gamma) * np.cos(alpha),
        np.cos(gamma) * np.sin(beta) * np.cos(alpha) +
        np.sin(gamma) * np.sin(alpha)
    ],
                        [
                            np.sin(gamma) * np.cos(beta),
                            np.sin(gamma) * np.sin(beta) * np.sin(alpha) +
                            np.cos(gamma) * np.cos(alpha),
                            np.sin(gamma) * np.sin(beta) * np.cos(alpha) -
                            np.cos(gamma) * np.sin(alpha)
                        ],
                        [
                            -np.sin(beta),
                            np.cos(beta) * np.sin(alpha),
                            np.cos(beta) * np.cos(alpha)
                        ]])

    # That is, lay the target image on the xy plane (the center of the image is O, so pay attention to the range of coordinates)
    # Through each pixel point, the coordinates of the intersections on the sphere are obtained, and the inverse transformation of the spherical projection corresponds to the pixels on the original image.
    for pix_proj in np.ndindex(arr_proj.shape[:2]):
        pix_img = projection(pix_proj, r, h_img, w_img, h_proj, w_proj)
        arr_proj[pix_proj] = arr_img[pix_img]

    Image.fromarray(arr_proj).show()  # annotate this line without pop-up
    # Image.fromarray(arr_proj).save(path_proj)  # annotate this line without outputting the file
